In [ ]:
# imports
import ibis
import ibis.selectors as s
import plotly.express as px

## local imports
from ihateai.data import read_training, transform
from ihateai.grid import InputOutputPair
from ihateai.utils import show_task_pairs, random_task_num

# configuration
px.defaults.template = "plotly_dark"

ibis.options.interactive = True
ibis.options.repr.interactive.max_rows = 10
ibis.options.repr.interactive.max_length = 3
ibis.options.repr.interactive.max_depth = 3
ibis.options.repr.interactive.max_columns = None

con = ibis.get_backend()

In [ ]:
t = read_training()
t

In [ ]:
t.schema()

In [ ]:
t.count()

In [ ]:
train = transform(t)
train

In [ ]:
test = transform(t, test=True)
test

In [ ]:
task_num, example_num = random_task_num(train)
grids = (
    train.filter(train["task_num"] == task_num)["input", "output"]
    .to_pyarrow()
    .to_pylist()[0]
)
iop = InputOutputPair(grids["input"], grids["output"])
iop

In [ ]:
iop.input == iop.input

In [ ]:
task_num = show_task_pairs(train, task_num=task_num)

In [ ]:
show_task_pairs(test, task_num=task_num)

In [ ]:
train.mutate(
    input_width_eq_output_width=train["input_width"] == train["output_width"],
    input_height_eq_output_height=train["input_height"] == train["output_height"],
).relocate(
    train.columns[:-2], "input_width_eq_output_width", "input_height_eq_output_height"
)

In [ ]:
train.group_by("task_num").agg(
    task_input_widths=train["input_width"].collect().unique().sort(),
    task_input_heights=train["input_height"].collect().unique().sort(),
    task_output_widths=train["output_width"].collect().unique().sort(),
    task_output_heights=train["output_height"].collect().unique().sort(),
).mutate(
    task_input_widths_same=ibis._["task_input_widths"].length() == 1,
    task_input_heights_same=ibis._["task_input_heights"].length() == 1,
    task_output_widths_same=ibis._["task_output_widths"].length() == 1,
    task_output_heights_same=ibis._["task_output_heights"].length() == 1,
).order_by("task_num").select(s.contains("_same")).value_counts().order_by(
    s.across(s.contains("_count"), ibis.desc)
)

In [ ]:
train.select(train["output_colors"].unnest()).distinct().count()

In [ ]:
test = transform(t, test=True)
test

In [ ]:
train.count()

In [ ]:
test.count()

In [ ]:
def decision_tree(t):
    res = (
        t.select(
            color_match=t["input_colors"] == t["output_colors"],
            grid_match=(
                (t["input_width"] == t["output_width"])
                & (t["input_height"] == t["output_height"])
            ),
            width_match=t["input_width"] == t["output_width"],
            height_match=t["input_height"] == t["output_height"],
        )
        .value_counts()
        .order_by("color_match", "grid_match", "width_match", "height_match")
        .mutate(
            percent=s.across(
                s.contains("count"),
                lambda x: 100 * x / x.sum(),
            )
        )
    )

    return res

In [ ]:
decision_tree(train)

In [ ]:
decision_tree(test)

In [ ]:
train = (
    train.join(
        train.select(
            "task_num",
            "example_num",
            train["input"].unnest(),
            train["output"].unnest(),
        )
        .group_by(
            "task_num",
            "example_num",
        )
        .agg(
            input_colors=ibis._["input"].collect().flatten().unique().sort(),
            output_colors=ibis._["output"].collect().flatten().unique().sort(),
        ),
        ["task_num", "example_num"],
    )
    .relocate(train.columns[:3], "input_colors", "output_colors")
    .drop(s.contains("_right"))
    .order_by("task_num", "example_num")
)
train

In [ ]:
train.count()

In [ ]:
train.select(
    color_match=train["input_colors"] == train["output_colors"],
    grid_match=(
        (train["input_width"] == train["output_width"])
        & (train["input_height"] == train["output_height"])
    ),
    input_match=(train["input_colors"].length() == train["output_colors"].length()),
    output_match=(train["input_colors"].length() == train["output_colors"].length()),
).value_counts().order_by("color_match", "grid_match", "input_match", "output_match")

In [ ]:
train["input"].unnest().as_table()["input"].unique().name("input_colors")

In [ ]:
train.group_by("input_width", "input_height", "output_width", "output_height").agg(
    count=train.count()
).order_by(ibis.desc("count"))

In [ ]:
test = t.select("task_num", t["test"].unnest()).select(
    "task_num",
    ibis._["test"]["input"],
    ibis._["test"]["output"],
)
test

In [ ]:
train.mutate(
    width=ibis._["input"].length(),
    height=ibis._["input"][0].length(),
).relocate

In [ ]:
train.count()

In [ ]:
test.count()

In [ ]:
test["task_num"].value_counts().order_by(ibis.desc("task_num_count"))

In [ ]:
test.filter(test["task_num"] == 398)